In [1]:
import numpy as np
from scipy.integrate import simps

# Model parameters
mean_reversion_rate = 0.05  # λ
long_run_average = 0.03     # η
volatility = 0.02          # θ
initial_rate = 0.02         # r(0)
time_horizon = 1.0          # T (years)
num_steps = 100             # Number of time steps

# Discretize the time horizon
dt = time_horizon / num_steps

# Simulate interest rates using Euler-Maruyama method
interest_rates = np.zeros(num_steps + 1)
interest_rates[0] = initial_rate

for i in range(1, num_steps + 1):
    drift = mean_reversion_rate * (long_run_average - interest_rates[i-1])
    diffusion = volatility * np.sqrt(interest_rates[i-1]) * np.random.normal(scale=np.sqrt(dt))
    interest_rates[i] = interest_rates[i-1] + drift * dt + diffusion

# Calculate average interest rate over time horizon
avg_rate = simps(interest_rates, dx=dt) / time_horizon

print("Simulated Interest Rates:")
print(interest_rates)
print("\nAverage Interest Rate:", avg_rate)


Simulated Interest Rates:
[0.02       0.02018491 0.02053109 0.02061337 0.02092568 0.02060592
 0.01991836 0.01950811 0.01954733 0.01956718 0.01942229 0.01925695
 0.01927601 0.01924113 0.01932979 0.01910602 0.01925414 0.01919494
 0.01933477 0.01970363 0.01991072 0.01992848 0.02011749 0.02035862
 0.02039704 0.02082302 0.0209372  0.02100884 0.02095143 0.02089321
 0.02118629 0.02099139 0.02056703 0.02044172 0.02053622 0.02019535
 0.020024   0.02005125 0.01985147 0.01969267 0.02007116 0.02016954
 0.0202519  0.02008496 0.0199767  0.02005007 0.02065046 0.02106343
 0.02113913 0.02105489 0.02071057 0.02067318 0.02067622 0.02075531
 0.02078743 0.02087303 0.02068811 0.02090943 0.02081335 0.02092723
 0.02130262 0.0216732  0.02156853 0.02140202 0.02180924 0.02144869
 0.02154094 0.02137362 0.0215518  0.0219218  0.02174577 0.02183779
 0.02162315 0.02178068 0.02159197 0.02158122 0.02087446 0.02075222
 0.02054741 0.02110354 0.02103468 0.02123274 0.0214225  0.02142968
 0.0215725  0.02143081 0.02162968 0.